In [1]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

Befor runing this code: 
1) create a .csv file containing a 'species_header' like 'species', 'names', etc. this column should contain the name of the species (or genuses by default) resolved by Open Tree of Life.
2) creta an empty folder (input_folder) and put the .csv file inside! 

That's all

DEFINE THE FOLLOWING PARAMETERS

In [72]:
#Paths
# Input folder containing CSV files
input_folder = '/mnt/c/Users/quirosgu/Desktop/RSV/Literature/' #/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/V1V2/'

species_header = 'full_species'#'query_otol_species' #put the column of your species
genus_header = 'genus'

LOTUSDB = '/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'
LOTUSDB_rc = '/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

FIRST RECOVER THE Q CODES FROM WIKIDATA AND UPDATE THE METADATA FILE

In [68]:
# Function to process a single CSV file and retrieve Q codes
def process_csv_file(input_file, output_folder):
    # Read the original CSV file containing the species names
    df_species = pd.read_csv(input_file)
    
    # Define the Wikidata Query Service endpoint URL
    endpoint_url = "https://query.wikidata.org/sparql"
    
    # Initialize an empty list to store the Q codes
    q_codes = []
    
    # Initialize a placeholder value for cases where no Q code is found
    placeholder_value = "Not Found"
    
    # Define the number of requests allowed per minute (adjust according to Wikidata's rate limits)
    requests_per_minute = 30
    
    # Iterate through the species names in the CSV file
    for index, row in df_species.iterrows():
        species_name = row[species_header]
    
        # Check if the species_name is a valid string
        if isinstance(species_name, str):
            # Query Wikidata to retrieve the corresponding Q code for the species
            sparql_query = f"""
            SELECT ?species ?speciesLabel WHERE {{
                
                ?species rdfs:label "{species_name}"@en.
                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            }}
            """
    
            response = requests.get(endpoint_url, headers={'Accept': 'text/csv'}, params={'query': sparql_query})
    
            if response.status_code == 200:
                csv_data = response.text
                df_qcode = pd.read_csv(StringIO(csv_data))
    
                if not df_qcode.empty:
                    q_code = df_qcode['species'].values[0]  # Extract the Q code
                    q_codes.append(q_code)  # Append the Q code to the list
                    print(f"Retrieved Q code {q_code} for species: {species_name}")
                else:
                    q_codes.append(placeholder_value)  # Add a placeholder value
                    print(f"No Q code found for species: {species_name}")
            else:
                q_codes.append(placeholder_value)  # Add a placeholder value for failed requests
                print(f"Failed to retrieve Q code for species: {species_name}. Status code: {response.status_code}")
    
            # Implement rate limiting by waiting between requests
            time.sleep(60 / requests_per_minute)  # Wait for one minute divided by the allowed requests per minute
        else:
            q_codes.append(placeholder_value)  # Add a placeholder value for invalid species names
    
    # Add the Q codes hyperlinks to the original CSV with a new column
    df_species['wikidata_Qcode_hyperlink'] = q_codes
    
    # Add the Q codes only to the original CSV with a new column
    df_species['wikidata_Qcode_species'] = df_species['wikidata_Qcode_hyperlink'].str.rsplit('/', n=1).str[-1]
    
    # Save the updated CSV file with Q codes in the output folder
    output_csv_filename = os.path.join(output_folder, os.path.basename(input_file))
    df_species.to_csv(output_csv_filename, index=False)
    print(f"Saved updated CSV with Q codes to {output_csv_filename}")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all CSV files in the input folder and process each one
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)
        process_csv_file(input_file, output_folder)

#once finished... check the lines for 'NOT FOUND' manually

Retrieved Q code http://www.wikidata.org/entity/Q35625 for species: Zingiber officinale
No Q code found for species: Garcinia mangostana
No Q code found for species: Garcinia mangostana
Retrieved Q code http://www.wikidata.org/entity/Q310050 for species: Phyllanthus emblica
Retrieved Q code http://www.wikidata.org/entity/Q311710 for species: Eurycoma longifolia
Retrieved Q code http://www.wikidata.org/entity/Q311710 for species: Eurycoma longifolia
No Q code found for species: Artemisia dracunculus
Retrieved Q code http://www.wikidata.org/entity/Q206733 for species: Curcuma zedoaria
Retrieved Q code http://www.wikidata.org/entity/Q11181633 for species: Ziziphus jujuba
Retrieved Q code http://www.wikidata.org/entity/Q11181633 for species: Ziziphus jujuba
Retrieved Q code http://www.wikidata.org/entity/Q11181633 for species: Ziziphus jujuba
Retrieved Q code http://www.wikidata.org/entity/Q11181633 for species: Ziziphus jujuba
Retrieved Q code http://www.wikidata.org/entity/Q11181633 for 

NOW, LETS USE THE QCODES TO RECOVER ALL THE COMPOUNDS (WITH REFERENCES) PRESENT IN EACH QCODE AND SAVE THEM AS INDIVIDUAL CSV FILES FROM LOTUS_DB FROZEN METADATA

In [74]:
# Load the LOTUSDB CSV
lotusdb_df = pd.read_csv(LOTUSDB, low_memory=False)

# Iterate through all CSV files in the input folder
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(output_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for q_code in df_species['wikidata_Qcode_species']:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if q_code == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['wikidata_Qcode'] == q_code]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            #create a 'chemical_superclass' column
            grouped_df['chemical_superclass'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_03class']
            # Save the grouped data as a TSV file with the Q code as the filename
            output_subfolder = os.path.join(output_folder, 'species_data')
            os.makedirs(output_subfolder, exist_ok=True)
            output_filename = os.path.join(output_subfolder, f"{q_code}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for Q code {q_code} to {output_filename}")


Saved grouped data for Q code Q35625 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q35625.tsv
Saved grouped data for Q code Q4733116 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q4733116.tsv
Saved grouped data for Q code Q4733116 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q4733116.tsv
Saved grouped data for Q code Q310050 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q310050.tsv
Saved grouped data for Q code Q311710 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q311710.tsv
Saved grouped data for Q code Q311710 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q311710.tsv
Saved grouped data for Q code Q155814) to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q155814).tsv
Saved grouped data for Q code Q206733 to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/species_data/Q206733.tsv
Saved groupe

In [75]:
# Create a subfolder for genus data
genus_subfolder = os.path.join(output_folder, 'genus_data')
os.makedirs(genus_subfolder, exist_ok=True)

# Iterate through all CSV files in the input folder
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(output_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for genus in df_species[genus_header]:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if genus == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['organism_taxonomy_08genus'] == genus]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data at the genus level based on 'genus_header' column
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            # Create a 'chemical_superclass' column
            grouped_df['chemical_superclass'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_03class']
            
            # Save the grouped data as a TSV file in the genus_data subfolder
            output_filename = os.path.join(genus_subfolder, f"{genus}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for genus {genus} to {output_filename}")


Saved grouped data for genus Zingiber to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Zingiber.tsv
Saved grouped data for genus Allophylus to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Allophylus.tsv
Saved grouped data for genus Allophylus to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Allophylus.tsv
Saved grouped data for genus Phyllanthus to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Phyllanthus.tsv
Saved grouped data for genus Eurycoma to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Eurycoma.tsv
Saved grouped data for genus Eurycoma to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Eurycoma.tsv
Saved grouped data for genus Artemisia to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Artemisia.tsv
Saved grouped data for genus Curcuma to /mnt/c/Users/quirosgu/Desktop/RSV/Literature/output_data/genus_data/Curcuma.tsv
Saved grou

RECOVER THE FRECUENCE BY CHEMICAL CLASS AND ADDED TO THE GENERAL TABLE

In [76]:
# Step 1: Initialize variables to store data
total_compounds = {}
chemical_classes = {}
chemical_superclasses = {}
references = {}
hyperlinks = {}

# Step 2: Iterate through .tsv files in the folder

species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        qcode = filename.split(".")[0]  # Extract Qcode from the filename

        # Load the .tsv file into a DataFrame
        df_compounds = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')

        # Step 3: Calculate frequencies of chemical classes for each Qcode (excluding 'not classified')
        filtered_classes = df_compounds[df_compounds['structure_taxonomy_npclassifier_03class'] != 'Not Classified']
        grouped = filtered_classes['structure_taxonomy_npclassifier_03class'].value_counts().reset_index()
        frequencies = grouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_03class']}", axis=1)
        chemical_classes[qcode] = "|".join(frequencies)

        # Step 4: Calculate frequencies of chemical superclasses for each Qcode (excluding 'not classified')
        filtered_sclasses = df_compounds[df_compounds['structure_taxonomy_npclassifier_02superclass'] != 'Not Classified']
        sgrouped = filtered_sclasses['structure_taxonomy_npclassifier_02superclass'].value_counts().reset_index()
        frequencies = sgrouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_02superclass']}", axis=1)
        chemical_superclasses[qcode] = "|".join(frequencies)
        chemical_classes[qcode] = "|".join(frequencies)
        
        # Step 5: Combine references and hyperlinks
        references[qcode] = "|".join(df_compounds['reference_wikidata'].dropna())
        hyperlinks[qcode] = "|".join(df_compounds['reference_doi'].dropna())

        # Step 6: Calculate the total number of reported compounds
        total_compounds[qcode] = len(df_compounds)

# Step 7: Load the general info .csv into a DataFrame
csv_file = None
for filename in os.listdir(output_folder):
    if filename.endswith(".csv"):
        csv_file = os.path.join(output_folder, filename)
        break  # Stop searching after finding the first CSV file

if csv_file is not None:
    # Load the CSV file into the df_general_info DataFrame
    df_general_info = pd.read_csv(csv_file)

# Step 7: Create new columns for combined information
df_general_info['predicted_class'] = df_general_info['wikidata_Qcode_species'].map(chemical_classes)
df_general_info['predicted_superclass'] = df_general_info['wikidata_Qcode_species'].map(chemical_superclasses)
#df_general_info['references'] = df_general_info['wikidata_Qcode'].map(references)
#df_general_info['referenceLabel'] = df_general_info['wikidata_Qcode'].map(hyperlinks)

# Step 8: Create a new column for total compounds
df_general_info['total_compounds'] = df_general_info['wikidata_Qcode_species'].map(total_compounds)

# Step 7: Save the updated DataFrame to the .csv file

# Replace values matching the pattern with an empty cell
df_general_info['predicted_class'] = df_general_info['predicted_class'].replace('index|predicted_class', '')
df_general_info['predicted_superclass'] = df_general_info['predicted_superclass'].replace('index|predicted_superclass', '')
output_csv_file = input_folder + 'Full_results.csv'


#add reported compounds to each Q code

LotusDB_rc = pd.read_csv(LOTUSDB_rc)

df = pd.merge(df_general_info, LotusDB_rc[['wikidata_Qcode', 'Reported_comp_Species']],
            how= 'left', left_on='wikidata_Qcode_species', right_on='wikidata_Qcode').drop_duplicates(subset=['wikidata_Qcode_species'])
df.drop('wikidata_Qcode', axis=1, inplace=True)

df = pd.merge(df, LotusDB_rc[['organism_taxonomy_08genus', 'Reported_comp_Genus']],
            how= 'left', left_on='genus', right_on='organism_taxonomy_08genus').drop_duplicates(subset=['full_species'])
df.drop('organism_taxonomy_08genus', axis=1, inplace=True)

df = pd.merge(df, LotusDB_rc[['organism_taxonomy_06family', 'Reported_comp_Family']],
            how= 'left', left_on='family', right_on='organism_taxonomy_06family').drop_duplicates(subset=['full_species'])
df.drop('organism_taxonomy_06family', axis=1, inplace=True)

df.to_csv(output_csv_file, index=False, sep =',')

Generate une plot by species (samples) and the chemical class

In [79]:
# Define base colors for each Pathway

pathway_colors = {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_colors.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades


# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['chemical_superclass'] != 'API Error-API Error') & 
                    (all_data['chemical_superclass'] != 'Not Classified-Not Classified')]

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Step 6: Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y=agg_data['species'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Superclasses Ocurrence for Species',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             category_orders={'chemical_superclass': sorted(agg_data['chemical_superclass'].unique())},
             orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species.html')

# Show the figure
fig.show()


/tmp/ipykernel_1617/1116008366.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/1116008366.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/1116008366.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Generate une plot by species (samples) and the Pathway

In [80]:
# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_01pathway'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)
# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'Pathway']).size().reset_index(name='recurrence')

# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y=agg_data['species'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Pathways Ocurrence for Species',
             labels={'recurrence': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'Pathway': sorted(agg_data['Pathway'].unique())},
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='Species')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot.html')

# Show or save the figure
fig.show()


/tmp/ipykernel_1617/2858114341.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/2858114341.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/2858114341.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Generate une plot by GENUS and the chemical class

In [81]:
# Define base colors for each Pathway

pathway_colors = {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_colors.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades


# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_08genus', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
genus_data_folder = os.path.join(output_folder, 'genus_data')
for filename in os.listdir(genus_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(genus_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_08genus': 'genus'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['chemical_superclass'] != 'API Error-API Error') & 
                    (all_data['chemical_superclass'] != 'Not Classified-Not Classified')]

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['genus', 'chemical_superclass']).size().reset_index(name='recurrence')

# Step 6: Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y=agg_data['genus'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Superclasses Ocurrence for Genus',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             category_orders={'chemical_superclass': sorted(agg_data['chemical_superclass'].unique())},
             orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Genus<i>')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1600)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_genus.html')

# Show the figure
fig.show()


/tmp/ipykernel_1617/1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Generate une plot by Genus and the Pathway

In [82]:
# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_08genus', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
genus_data_folder = os.path.join(output_folder, 'genus_data')
for filename in os.listdir(genus_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(genus_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_08genus': 'genus'}, inplace=True)
all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['genus', 'Pathway']).size().reset_index(name='recurrence')

# Sort the DataFrame by 'genus' alphabetically
agg_data = agg_data.sort_values(by='genus')

# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y=agg_data['genus'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Pathways Ocurrence for Genus',
             labels={'recurrence': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'genus': sorted(agg_data['genus'].unique())},  # Sort 'genus' alphabetically,
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='Genus')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1200, height=1400)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot_genus.html')

# Show or save the figure
fig.show()


/tmp/ipykernel_1617/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1617/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.